# Doppler testing notebook

In this notebook, we want to extract a visible Doppler shift from captured signals.

## SHARP

To start off, some general insights about SHARP:

- The main idea of SHARP is to perform a phase sanitization based on a path resolution
- Instead of requiring multiple antennas, the idea is to decompose the signal into its different multipath components
- This is done using a "query" or compressive sensing based optimization procedure to search for frequency/path-delay weights
- The procedure is repeated twice; once for coarse, once for a finer search
- The strongest path is then used to remove phase errors from the others

The resulting signal can then be processed with the usual STFT to reveal the Doppler frequency components.

- The window over which the STFT happens must be small enough to guarantee for constant velocity within
- The original paper padded windows of size $31$ with zeros to achieve a better spectral resolution (artificially smoothed)

## TODOs

- [x] Plot Doppler from theoretical channel model (i.e. the linear precoding CSI mask)
- [x] Plot Amplitude/Phase
- [x] Create a lineplot
- [x] Take a longer trace and maybe start from a larger t: Didnt change much
- [x] Shorter interframe spacing: Nothing new to see
- [x] Take "faster" data: Improved things. 0 to 100m over the time works quite nicely 
- [x] Flip data mask (time reverse): Revealed that edge effect is in fact part of the data
- [x] Check phases after phase sanitization: Looks fairly random, no insights gained
- [x] Do phase-unwrapping in time: Didnt change much
- [ ] Doppler without sanitization
- [ ] Split up sanitization and doppler computation
- [ ] Check the shape of phases (vs subcarrier): Why does simple phase sanitization not remove the linear doppler effect?
- [ ] Search for paper and send to Francesca about the linear phase sanitization

In [ ]:
import numpy as np
import scipy
from scipy.fftpack import fftshift
from scipy.signal import stft
from pathlib import Path

import cvxpy as cp
import osqp
from dataclasses import dataclass
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

from sensession import Database
from sensession.config import APP_CONFIG, configure_logger
from sensession.campaign.processor import CampaignProcessor

from IPython.display import display

APP_CONFIG.loglevel = "WARNING"
configure_logger()

## Configuration options

The following are some notebook-wide configuration options. Mainly these include:

- The "search space" for the quadratic program
- Noise levels
- Subselection parameters (to not consider all subcarriers)
- FFT window sizes

In [ ]:
# subcarrier spacing
delta_f = 312.5e3

# Instead of using every subcarrier in the T search matrix, we can also use
# just a subset of them.
subcarrier_spacing = 1

# range of multipath delays to "search" for by constructing T
t_min = -3e-7
t_max = 5e-7
delta_t = 1e-7

# After a "coarse" search, do a finer search around the result
range_refined_up = 3.5e-7
range_refined_down = 3e-7
delta_t_refined = 5e-9

# Noise levels to cut off when plotting spectrogram
noise_min = -25
noise_max = 0

# optimization lib
optimization_algo = "osqp"  # or cvx

# FFT/STFT config
stft_window = 100  # window size
stft_window_step = 1  # How far to advance the window
n_fft = 100  # FFT size

# Loading and preparing data

Load some prepared data

In [ ]:
# Example session to choose
ex_session_num = 0

# ---------------------------------------------------------------
# -------- Preprocess the DataFrame
with Database(Path("../../data/doppler_emulation"), lazy=False) as db:
    proc = (
        CampaignProcessor(
            db.get_csi(),
            db.get_meta(),
            lazy=False,
        )
        .unwrap()
        .filter("antenna_idxs", 0)
        .scale_magnitude()
        .detrend_phase(pin_edges=False)
    )

    csi_df = proc.csi
    meta_df = proc.meta


# -------- Extract a sample session
unique_ids = meta_df.unique("schedule_name", maintain_order=True)
ex_session_id = unique_ids.item(ex_session_num, "schedule_name")
example_df = csi_df.filter(pl.col("schedule_name") == ex_session_id)


# -------- Regroup dataframe CSI values
def regroup(df: pl.DataFrame):
    return df.group_by("capture_num", maintain_order=True).agg(
        pl.col("meta_id").first(),
        pl.col("timestamp").first(),
        pl.col("sequence_number").first(),
        "csi_abs",
        "csi_phase",
        "subcarrier_idxs",
        pl.col("rssi").first(),
        pl.col("antenna_rssi").first(),
        pl.col("collection_name").first(),
        pl.col("receiver_name").first(),
    )


example_df = regroup(example_df)
print(f"Loaded example data of shape: {example_df.shape}")

In [ ]:
display(example_df)

## Loading and using curves

For some Doppler experiments, we use random curves to emulate the distance dependency.

In [ ]:
# curves_file = "../data/linear_doppler_normal/curves.parquet"
# curves_df = pl.read_parquet(curves_file)

# # Choose an example curve to look at
# ex_curve_idx = example_df.item(0, "num_curve")


# # -------- Unwrap the corresponding curve
# unwrapped_curve = (
#     curves_df.filter(pl.col("num_curve") == ex_curve_idx)
#     .explode("curve")
#     .with_row_index("time_idx")
# )

# # --------------------------- Plot the baseline curve
# plt.figure(figsize=(10, 4))
# sns.lineplot(
#     data=unwrapped_curve,
#     x="time_idx",
#     y="curve",
# ).set(title=f"Sampled random curve (number {ex_curve_idx})")
# plt.show()

# Sharp Optimization Helpers

First we define a few helpers for the quadratic program that sharp solves. This code is mostly taken from the SHARP open source repository, with minor adaptations for performance and readability.

In [ ]:
def convert_to_complex(real_im_n: np.ndarray):
    """
    Convert stacked [real, complex] array back to complex
    """
    len_vect = real_im_n.shape[0] // 2
    return real_im_n[:len_vect] + 1j * real_im_n[len_vect:]


def build_T_matrix(frequency_vector, delta_t, t_lower, t_upper):
    """
    Build T matrix

    This matrix serves as "search" weights for the paths, by resolving
    different frequencies and path delays. The optimization procedure
    then finds which of these correlate highly with the data.

    Args:
        frequency_vector : The (baseband) frequencies to weigh with in Hz.
            Must match the subcarrier frequencies.
        delta_t : Path delay spacing (in seconds)
        t_lower : Minimal path delay (in seconds) to include in search
        t_upper : Maximum path delay (in seconds) to include in search
    """
    num_paths = int((t_upper - t_lower) / delta_t)

    # Times of the different paths
    times = t_lower + delta_t * np.arange(num_paths)
    T_matrix = np.exp(-2j * np.pi * np.outer(frequency_vector, times))
    return T_matrix, times


def lasso_regression_osqp(
    H_matrix: np.ndarray,
    T_matrix: np.ndarray,
    selected_subcarriers: np.ndarray,
):
    lambd = 1e-1

    # Auxiliary Data
    row_T = int(T_matrix.shape[0])
    col_T = T_matrix.shape[1]
    m = 2 * row_T
    n = 2 * col_T
    In = scipy.sparse.eye(n)
    Im = scipy.sparse.eye(m)
    On = scipy.sparse.csc_matrix((n, n))
    Onm = scipy.sparse.csc_matrix((n, m))
    q_vec = np.zeros(2 * n + m)
    A2 = scipy.sparse.hstack([In, Onm, -In])
    A3 = scipy.sparse.hstack([In, Onm, In])
    ones_n_matr = np.ones(n)
    zeros_n_matr = np.zeros(n)
    zeros_nm_matr = np.zeros(n + m)

    # ------------------------------------------------------------
    T_real = np.real(T_matrix[selected_subcarriers, :])
    T_imag = np.imag(T_matrix[selected_subcarriers, :])

    # Construct T_matrix_real by stacking real and imaginary parts
    T_matrix_real = np.vstack(
        (
            np.hstack((T_real, -T_imag)),
            np.hstack((T_imag, T_real)),
        )
    )

    H_matrix_real = np.hstack(
        (
            np.real(H_matrix[selected_subcarriers]),
            np.imag(H_matrix[selected_subcarriers]),
        )
    )

    # OSQP data
    A_mat = scipy.sparse.vstack(
        [scipy.sparse.hstack([T_matrix_real, -Im, Onm.T]), A2, A3], format="csc"
    )
    P_mat = scipy.sparse.block_diag([On, Im, On], format="csc")

    l_vec = np.hstack([H_matrix_real, -np.inf * ones_n_matr, zeros_n_matr])
    u_vec = np.hstack([H_matrix_real, zeros_n_matr, np.inf * ones_n_matr])

    # Set up OSQP problem
    prob = osqp.OSQP()
    prob.setup(P_mat, q_vec, A_mat, l_vec, u_vec, warm_start=True, verbose=False)

    # Update linear cost
    q_new = np.hstack([zeros_nm_matr, lambd * ones_n_matr])
    prob.update(q=q_new)

    # Solve and take the first n components
    res = prob.solve()
    x_out = res.x[:n]

    r_opt = convert_to_complex(x_out)
    return r_opt

In [ ]:
def lasso_regression_cvx(H_matrix, T_matrix, selected_subcarriers):
    """
    Same optimization procedure as above, but with cvx.

    NOTE: This is much slower, but otherwise plug & play as well.
    """
    T_matrix_pilot = T_matrix[selected_subcarriers]
    H_matrix_pilot = H_matrix[selected_subcarriers]

    s_0 = cp.Variable(T_matrix_pilot.shape[1] * 2)
    lambd = cp.Parameter(nonneg=True)
    lambd.value = 1e-1
    objective = cp.Minimize(
        cp.power(
            cp.norm(H_matrix_pilot - T_matrix_pilot @ convert_to_complex(s_0), 2), 1
        )
        + lambd * cp.norm(convert_to_complex(s_0), 1)
    )
    problem = cp.Problem(objective)
    problem.solve()
    s_est = convert_to_complex(s_0.value)
    return s_est

# SHARP phase sanitization

Next up, we define the precise phase sanitization feature as proposed by SHARP. Also mostly taken from the SHARP repository and adapted for performance and readability.

In [ ]:
def interpolate_phase(H_est: np.ndarray):
    phase_before = np.unwrap(np.angle(H_est.T), axis=0)
    ones_vector = np.ones((2, phase_before.shape[0]))
    ones_vector[1, :] = np.arange(0, phase_before.shape[0])
    for tidx in range(1, phase_before.shape[1]):
        stop = False
        idx_prec = -1
        while not stop:
            phase_err = phase_before[:, tidx] - phase_before[:, tidx - 1]
            diff_phase_err = np.diff(phase_err)
            idxs_invert_up = np.argwhere(diff_phase_err > 0.9 * np.pi)[:, 0]
            idxs_invert_down = np.argwhere(diff_phase_err < -0.9 * np.pi)[:, 0]
            if idxs_invert_up.shape[0] > 0:
                idx_act = idxs_invert_up[0]
                if idx_act == idx_prec:  # to avoid a continuous jump
                    stop = True
                else:
                    phase_before[idx_act + 1 :, tidx] = (
                        phase_before[idx_act + 1 :, tidx] - 2 * np.pi
                    )
                    idx_prec = idx_act
            elif idxs_invert_down.shape[0] > 0:
                idx_act = idxs_invert_down[0]
                if idx_act == idx_prec:
                    stop = True
                else:
                    phase_before[idx_act + 1 :, tidx] = (
                        phase_before[idx_act + 1 :, tidx] + 2 * np.pi
                    )
                    idx_prec = idx_act
            else:
                stop = True
    for tidx in range(1, H_est.shape[1] - 1):
        val_prec = phase_before[:, tidx - 1 : tidx]
        val_act = phase_before[:, tidx : tidx + 1]
        error = val_act - val_prec
        temp2 = np.linalg.lstsq(ones_vector.T, error, rcond=None)[0]
        phase_before[:, tidx] = phase_before[:, tidx] - (np.dot(ones_vector.T, temp2)).T

    return np.abs(H_est) * np.exp(1j * phase_before.T)


def sanitize_phase(csi_ts: np.ndarray) -> np.ndarray:
    """
    Sanitize phases:

    Resolve multiple paths from an optimization problem; Since all paths are
    affected by the same phase shifts, we may use e.g. the strongest path to
    remove the phase shifts.
    """
    opt_func = (
        lasso_regression_osqp if optimization_algo == "osqp" else lasso_regression_cvx
    )

    (F_frequency,) = csi_ts.shape

    # Construct frequency vector
    # must represent the frequencies of the employed subcarriers
    # Since the tools don't report DC subcarriers, we need to avoid it as well
    freq_p = delta_f * np.arange(F_frequency // 2) + delta_f
    freq_n = freq_p - (delta_f * F_frequency / 2)
    frequency_vector = np.concatenate((freq_n, freq_p))

    start_subcarrier = 0
    end_subcarrier = frequency_vector.shape[0]
    select_subcarriers = np.arange(start_subcarrier, end_subcarrier, subcarrier_spacing)

    # -------- First step: Coarse search
    T_matrix, time_matrix = build_T_matrix(frequency_vector, delta_t, t_min, t_max)

    complex_opt_r = opt_func(
        csi_ts,
        T_matrix,
        select_subcarriers,
    )

    position_max_r = np.argmax(abs(complex_opt_r))
    time_max_r = time_matrix[position_max_r]

    # -------- Second step: Finer search
    T_matrix_refined, _ = build_T_matrix(
        frequency_vector,
        delta_t_refined,
        max(time_max_r - range_refined_down, t_min),
        min(time_max_r + range_refined_up, t_max),
    )

    complex_opt_r_refined = opt_func(
        csi_ts,
        T_matrix_refined,
        select_subcarriers,
    )
    position_max_r_refined = np.argmax(abs(complex_opt_r_refined))

    # -------- Final step: compute the resulting value

    # This part is changed from the original SHARP; It should allow for an
    # interpolation of the missing DC value
    freq_vec = delta_f * np.arange(-F_frequency // 2, F_frequency // 2 + 1, 1)
    T_matrix, time_matrix = build_T_matrix(
        freq_vec,
        delta_t_refined,
        max(time_max_r - range_refined_down, t_min),
        min(time_max_r + range_refined_up, t_max),
    )

    Tr = T_matrix * complex_opt_r_refined
    return np.sum(Tr * np.conj(Tr[:, position_max_r_refined][:, np.newaxis]), axis=1)


###############################################################################
def sanitize_csi_phase(csi_ts: np.ndarray) -> np.ndarray:
    assert csi_ts.ndim == 2, (
        f"Expecting a timeseries (2d array) of CSI values, got {csi_ts.ndim}"
    )
    print(f"Phase-Sanitizing CSI matrix of shape: {csi_ts.shape}")
    return np.apply_along_axis(sanitize_phase, axis=1, arr=csi_ts)

# SHARP Doppler

With the phase sanitizied, we can find the Doppler Spectrum using an STFT.

In [ ]:
def get_doppler_spectrum(
    csi_data: np.ndarray,
    window_size: int = 100,
    window_step_size: int = 1,
    n_fft: int = 100,
) -> np.ndarray:
    """
    Calculate STFT in windows

    Args:
        csi_data: (num_samples, num_subcarrier) complex-valued CSI array
        window_size: STFT window size
        window_step_size: Sliding window step size
        n_fft: Number of FFT bins in each step

    Returns:
        A (num_fft, num_samples + 1) doppler spectrogram
    """
    assert window_size >= window_step_size, "Window size must be greater than step size"

    _, _, spectrum = stft(
        csi_data,
        window="hann",
        # nperseg=window_size,
        # noverlap=window_size - window_step_size,
        # nfft=n_fft,
        nperseg=200,
        noverlap=200 - 1,
        nfft=200,
        return_onesided=False,
        axis=0,
    )

    spectrum = fftshift(spectrum, axes=0)
    spectrum = np.abs(spectrum * np.conj(spectrum)).sum(axis=1)

    # TEMP remove
    # noise_lev = -1.5
    # t = 10**noise_lev
    # spectrum[spectrum < t] = t

    print(f"Calculated Doppler Spectrum of shape: {spectrum.shape}")
    # Return dB-converted
    return 10 * np.log10(spectrum / np.max(spectrum))


@dataclass
class DopplerPlotConfig:
    spectrum: np.ndarray  # Doppler Spectrum
    fig_title: str


def plot_doppler(
    configs: list[DopplerPlotConfig], label_size: int = 12, title_size: int = 16
):
    """
    Plot Doppler spectrum

    Args:
        spectrum   : A (num_packets, num_doppler_bins) doppler spectrum array
        title      : Figure title
        label_size : Fontsize of axis labels
        title_size : Fontsize of figure title
    """

    # Assuming 5 subplots
    if isinstance(configs, list):
        lcfgs = len(configs)
        assert lcfgs == 8, f"Doppler plot hardcoded for 5 or 1 receiver(s), got {lcfgs}"
        fig, axes = plt.subplots(
            figsize=(14, 8), nrows=4, ncols=2, sharex=True, sharey=True
        )
        # needed for uneven counts
        # axes = axes[:5]
        # fig.delaxes(axes[2, 1])
    else:
        fig, ax = plt.subplots(figsize=(12, 7))
        axes = np.array([ax])
        configs = [configs]

    for ax, config in zip(axes.flat, configs):
        if "iwl5300" in config.fig_title:
            ax.axvline(x=1373, color="green", linestyle="--")  # Vertical green line
            ax.axvline(
                x=1373 + 25, color="y", linestyle="--", linewidth=2
            )  # Vertical green line
            ax.axvline(
                x=1373 - 25, color="y", linestyle="--", linewidth=2
            )  # Vertical green line

        sns.heatmap(config.spectrum, ax=ax)  # , vmin=noise_min, vmax=noise_max)
        ax.set_xlabel("Packet number", size=label_size)
        ax.set_ylabel(r"$v_p \cos \alpha_p$ [m/s]", size=label_size)
        ax.grid(False)
        ax.set_title(config.fig_title, size=title_size)

    fig.tight_layout()
    plt.show()


def plot_doppler_old(H, window_size: int = 100, n_fft: int = 100):
    from scipy.signal.windows import hann
    from scipy.fftpack import fft
    from scipy.fftpack import fftshift

    n_symbols = 46
    half_symbols = int(n_symbols // 2)
    csi_matrix = np.zeros(shape=((H.shape[0] + n_symbols), H.shape[1]), dtype=complex)
    csi_matrix[half_symbols : csi_matrix.shape[0] - half_symbols] = H
    H_Doppler = np.zeros(shape=((H.shape[0] + n_symbols), window_size))

    for i in range(half_symbols, int(csi_matrix.shape[0] - n_symbols / 2)):
        csi_matrix_cut = csi_matrix[i - half_symbols : i + half_symbols, :]
        csi_matrix_cut = np.nan_to_num(csi_matrix_cut)

        hann_window = np.expand_dims(hann(n_symbols), axis=-1)
        csi_matrix_wind = np.multiply(csi_matrix_cut, hann_window)
        csi_doppler_prof = fft(csi_matrix_wind, n=n_fft, axis=0)
        csi_doppler_prof = fftshift(csi_doppler_prof, axes=0)

        csi_d_map = np.abs(csi_doppler_prof * np.conj(csi_doppler_prof))
        csi_d_map = np.sum(csi_d_map, axis=1)
        H_Doppler[i] = csi_d_map

    H_Doppler = H_Doppler[half_symbols : csi_matrix.shape[0] - half_symbols]

    H_Doppler = H_Doppler / np.max(H_Doppler)
    H_Doppler = 10 * np.log10(H_Doppler).T

    fig, ax = plt.subplots()
    X, Y = np.meshgrid(range(H_Doppler.shape[1] + 1), range(H_Doppler.shape[0] + 1))
    ax.set_xlabel("Packet number", size=16)
    ax.set_ylabel(r"$v_p \cos \alpha_p$ [m/s]", size=14)
    ax.pcolormesh(X, Y, H_Doppler, cmap="viridis", vmin=noise_min, vmax=noise_max)
    plt.show()

# Numpy conversion and phase sanitization

We start by converting the data to numpy arrays and applying the SHARP phase sanitization procedure.

In [ ]:
def col_to_numpy(df: pl.DataFrame, column: str) -> np.ndarray:
    return np.array(df.get_column(column).to_list())


def to_csi(df: pl.DataFrame) -> np.ndarray:
    return col_to_numpy(df, "csi_abs") * np.exp(1j * col_to_numpy(df, "csi_phase"))


def df_to_csi_array_dict(df: pl.DataFrame):
    return {
        receiver_name[0]: to_csi(group)
        for receiver_name, group in df.group_by(["receiver_name"], maintain_order=True)
    }


def df_to_sequence_dict(df: pl.DataFrame):
    return {
        receiver_name[0]: col_to_numpy(group, "sequence_number")
        for receiver_name, group in df.group_by("receiver_name", maintain_order=True)
    }


def phase_sanitize_dict_array(d: dict):
    return {receiver_name: sanitize_csi_phase(csi) for receiver_name, csi in d.items()}


def phase_interp_dict_array(d: dict):
    return {receiver_name: interpolate_phase(csi) for receiver_name, csi in d.items()}


# First extract numpy arrays for the idle and active sessions
idle_df = example_df.filter(pl.col("collection_name").str.contains("warmup"))
idle_np_csi = df_to_csi_array_dict(idle_df)
idle_np_seq = df_to_sequence_dict(idle_df)

acti_df = example_df.filter(~pl.col("collection_name").str.contains("warmup"))

# indices = np.random.choice(np.arange(2500), size=1250, replace=False)
# acti_df = acti_df.filter(pl.col("sequence_number").is_in(indices))

# Sort the indices in ascending order
# ordered_indices = np.sort(indices)

acti_np_csi = df_to_csi_array_dict(acti_df)
acti_np_seq = df_to_sequence_dict(acti_df)

# Perform sanitization
# idle_np_csi_san = phase_sanitize_dict_array(idle_np_csi)
# acti_np_csi_san = phase_sanitize_dict_array(acti_np_csi)

# Perform SHARP-phase interpolation
# idle_np_csi_interp = phase_interp_dict_array(idle_np_csi_san)
# acti_np_csi_interp = phase_interp_dict_array(acti_np_csi_san)

# Alternative direct phase preprocessing

In [ ]:
from scipy.interpolate import CubicSpline


def remove_outliers(
    channel: np.ndarray, window_size: int = 10, threshold: float = 3.0
) -> np.ndarray:
    """
    Remove strong outliers in timeseries CSI data using sliding window Z-score detection.

    Args:
        channel: (n_samples, n_subcarrier) complex valued channel timeseries (CSI).
        window_size: Number of samples before and after the current sample to consider for local statistics.
        threshold: Z-score threshold for outlier detection.

    Returns:
        np.ndarray: Channel data with outliers removed.
    """
    n_samples = channel.shape[0]
    magnitudes = np.abs(channel).mean(axis=1)  # Compute magnitudes (n_samples,)

    # Initialize mask for non-outliers
    non_outlier_mask = np.ones(n_samples, dtype=bool)

    # Sliding window Z-score detection
    for i in range(n_samples):
        # Define window range
        start = max(0, i - window_size)
        end = min(n_samples, i + window_size + 1)

        # Exclude the current sample from the window for unbiased stats
        window_magnitudes = np.concatenate(
            (magnitudes[start:i], magnitudes[i + 1 : end])
        )

        # Compute local mean and std
        local_mean = np.mean(window_magnitudes)
        local_std = np.std(window_magnitudes)

        # If std is very small (e.g., constant values), skip outlier detection to avoid division by zero
        if local_std == 0:
            continue

        # Compute Z-score for the current sample
        z_score = (magnitudes[i] - local_mean) / local_std

        # Mark as outlier if Z-score exceeds threshold
        if np.abs(z_score) > threshold:
            non_outlier_mask[i] = False

    # Print the number of removed points
    num_removed = np.sum(~non_outlier_mask)
    print(f"Removed {num_removed} outlier(s) out of {n_samples} total points.")

    # Return the filtered data
    return channel[non_outlier_mask]


def impute(
    channel: np.ndarray, sequence_nums: np.ndarray, max_sequence: int
) -> np.ndarray:
    """
    Impute missing sequence numbers in timeseries CSI data using cubic spline interpolation.

    Args:
        channel: (n_samples, n_subcarrier) complex valued channel timeseries (CSI).
        sequence_nums: (n_samples,) integer valued sequence number of the respective CSI
        max_sequence: The maximum expected sequence number

    Returns:
        np.ndarray: Channel with missing sequence numbers imputed
    """
    # Full range of expected sequence numbers
    full_sequence = np.arange(0, max_sequence + 1)

    # Initialize full channel with NaNs
    full_channel = np.full(
        (len(full_sequence), channel.shape[1]), np.nan + 1j * np.nan, dtype=complex
    )

    # Place existing data into full channel
    full_channel[sequence_nums] = channel

    # Identify missing sequence numbers
    missing_indices = np.setdiff1d(full_sequence, sequence_nums)

    # Interpolate amplitude and phase separately for each subcarrier
    for subcarrier in range(channel.shape[1]):
        # Extract amplitude and phase from provided channel
        original_amplitude = np.abs(channel[:, subcarrier])
        original_phase = np.unwrap(np.angle(channel[:, subcarrier]))

        # Fit cubic splines with extrapolation
        amp_spline = CubicSpline(sequence_nums, original_amplitude, extrapolate=True)
        phase_spline = CubicSpline(sequence_nums, original_phase, extrapolate=True)

        # Interpolate missing values
        imputed_amplitude = amp_spline(full_sequence)
        imputed_phase = phase_spline(full_sequence)

        # Only update missing indices in full_channel
        full_channel[missing_indices, subcarrier] = imputed_amplitude[
            missing_indices
        ] * np.exp(1j * imputed_phase[missing_indices])

    return full_channel

In [ ]:
def clean_phase(channel: np.ndarray, subcarrier_indices: np.ndarray) -> np.ndarray:
    """
    Clean the phase of the channel by applying a linear phase correction.

    Args:
        channel: (n_samples, n_subcarrier) complex-valued channel timeseries (CSI)
        subcarrier_indices: (n_subcarrier,) subcarrier index array mapping each index to a subcarrier number

    Returns:
        Phase-corrected channel of the same shape (n_subcarrier, n_samples) as input
    """
    # Extract phases from the channel matrix
    phases = np.angle(channel)
    phases = np.unwrap(phases, axis=1)  # Unwrap phase along the subcarrier axis

    # The correction assumes symmetric subcarrier indices, so we shift.
    subcarrier_indices -= np.mean(subcarrier_indices)

    # Compute linear correction for each sample
    delta_subcarrier = subcarrier_indices[-1] - subcarrier_indices[0]
    k = (phases[:, -1] - phases[:, 0]) / delta_subcarrier  # Slope of linear phase
    b = np.mean(phases, axis=1)  # Mean phase offset

    # Generate correction matrix
    correction = np.outer(subcarrier_indices, k) + b

    # Apply correction and return the phase-corrected channel
    return channel / np.exp(1j * correction.T)


# idle_np_csi_cleaned = {
#     recv: impute(csi, idle_np_seq[recv], 500) for recv, csi in idle_np_csi.items()
# }
# acti_np_csi_cleaned = {
#     recv: impute(csi, acti_np_seq[recv], 2500) for recv, csi in acti_np_csi.items()
# }

subcarrier_idxs = list(range(-28, 0)) + list(range(1, 29))
idle_np_csi_san = {
    recv: clean_phase(csi, subcarrier_idxs) for recv, csi in idle_np_csi.items()
}
acti_np_csi_san = {
    recv: clean_phase(csi, subcarrier_idxs) for recv, csi in acti_np_csi.items()
}

## Calculating Doppler Spectra for idle and emulated movement

Calculate the Doppler spectra!

In [ ]:
def get_all_dopplers_in_session(
    d: dict, stft_window: int = 100, stft_window_step: int = 1, n_fft: int = 100
) -> list[DopplerPlotConfig]:
    """
    Calculate all Doppler Spectra for receivers present in the DataFrame separately.

    NOTE: df must contain only one session!
    """
    return [
        DopplerPlotConfig(
            spectrum=get_doppler_spectrum(
                csi,
                window_size=stft_window,
                window_step_size=stft_window_step,
                n_fft=n_fft,
            ),
            fig_title=f"Doppler Spectrum {receiver_name}",
        )
        for receiver_name, csi in d.items()
    ]


idle_dopplers = get_all_dopplers_in_session(
    idle_np_csi_san,  # idle_np_csi_interp,
    stft_window=stft_window,
    stft_window_step=stft_window_step,
    n_fft=n_fft,
)
acti_dopplers = get_all_dopplers_in_session(
    acti_np_csi_san,  # acti_np_csi_interp,
    stft_window=stft_window,
    stft_window_step=stft_window_step,
    n_fft=n_fft,
)

In [ ]:
full_range = np.arange(0, 2500)
np.setdiff1d(full_range, acti_np_seq["iwl5300"])

# Idle Doppler

Plot the Doppler Shift of all different receivers from seeing the idle channel

In [ ]:
plot_doppler(idle_dopplers)

## Active Dopplers

Now plot the Doppler spectra extracted from an active channel (i.e. with something happening).

In [ ]:
plot_doppler(acti_dopplers)

In [ ]:
def explode(df: pl.DataFrame) -> pl.DataFrame:
    return df.explode(
        "subcarrier_idxs",
        "csi_abs",
        "csi_phase",
    )


def plot_receiver_timeseries(dataframe, key="csi_abs", figsize=(18, 22)):
    n_receiver = dataframe.select("receiver_name").n_unique()
    fig, axs = plt.subplots(nrows=n_receiver, figsize=figsize)

    if not isinstance(axs, np.ndarray):
        axs = [axs]

    for i, (receiver, group_df) in enumerate(
        dataframe.group_by(["receiver_name"], maintain_order=True)
    ):
        ax = axs[i]
        sns.lineplot(
            x="timestamp",
            y=key,
            hue="subcarrier_idxs",
            data=group_df.to_pandas(),
            legend="full",
            ax=ax,
        ).set(title=f"{receiver[0]}")
        ax.legend(loc="lower right")

    plt.tight_layout()
    plt.show()


example_subc = 3
example_subc_csi = explode(acti_df).filter(pl.col("subcarrier_idxs") == example_subc)

sns.set(font_scale=2)
plot_receiver_timeseries(example_subc_csi, "csi_abs", figsize=(18, 15))

In [ ]:
plot_receiver_timeseries(example_subc_csi, "csi_phase", figsize=(18, 15))

In [ ]:
fig = plt.figure(figsize=(18, 6))
linestyles = ["solid", "dashed", "solid", "dotted", "dashdot"]
for linestyle, (receiver_name, csi) in zip(linestyles, acti_np_csi.items()):
    angle = np.angle(csi)
    sns.lineplot(
        angle[:, example_subc + 32], label=f"{receiver_name}", linestyle=linestyle
    )
plt.legend()

# Simulated CSI

In an ideal world, the channel would be completely flat, i.e. we would directly measure the mask up to some factor. In other words, we can use the mask as simulated CSI values.

In [ ]:
from sensession.tools.tool import CsiGroup
from sensession.database import csigroup_to_dataframe


def linear_mask(num: int = 5000) -> np.ndarray:
    """
    Linear movement emulation
    """
    n_subcarrier = 64

    # this is f / c as an array across subcarriers
    freq_comb = 8.012 + np.arange(0, n_subcarrier, 1) * 0.0010424

    # linear distance: moving from 0 to 10m distance over the course of 700 packets
    # 700 packets should be between 1 and 5 seconds, depending on interframe space.
    distance = np.linspace(0.0, 100.0, num=num)

    # exp(-j 2 pi d(t) f / c)
    doppler_shift = np.exp(2j * np.pi * freq_comb[:, np.newaxis] * distance)

    # This should emulate a two-path model: 0.9 times the static (i.e. non-emulated)
    # component, then 0.1 times our doppler shifted one.
    return 0.8 + 0.2 * doppler_shift


def get_simulated_df() -> pl.DataFrame:
    num_data = 2500

    # must be (num_data, num_antenna, num_subcs) shape for the csigroup
    csi_data = linear_mask(num_data).T.reshape(num_data, 1, 64)

    group = CsiGroup(
        timestamps=list(range(num_data)),
        sequence_nums=list(range(num_data)),
        csi_vals=list(csi_data),
        rssi=np.zeros(num_data).astype(int).tolist(),
        antenna_rssi=[[0]] * num_data,
    )

    return csigroup_to_dataframe(group, meta_id="idealized_id")


simulated_data = get_simulated_df()
proc = (
    CampaignProcessor(
        simulated_data,
        pl.DataFrame(
            {
                "meta_id": "idealized_id",
                "antenna_idxs": [[0]],
                "subcarrier_idxs": [list(range(-32, 0)) + list(range(1, 33))],
                "collection_name": "simulated",
                "schedule_name": "simulated",
                "receiver_name": "god_almighty",
            }
        ),
        out_file="data/weak_fast_doppler/test.parquet",
    )
    .unwrap()
    .filter("antenna_idxs", 0)
    .remove_edge_subcarriers()
    .scale_magnitude()
)
simulated_data = proc.csi
simulated_data = regroup(simulated_data)

# Convert to numpy and perform sanitization
simulated_csi = df_to_csi_array_dict(simulated_data)
simulated_csi_san = phase_sanitize_dict_array(simulated_csi)
simulated_csi_interp = phase_interp_dict_array(simulated_csi_san)

In [ ]:
# Extract and plot Doppler
simulated_dopplers = get_all_dopplers_in_session(simulated_csi)[0]
plot_doppler(simulated_dopplers)

In [ ]:
example_idxs = [example_subc]
data = simulated_data.explode("csi_abs", "csi_phase", "subcarrier_idxs").filter(
    pl.col("subcarrier_idxs").is_in(example_idxs)
)
plot_receiver_timeseries(data, "csi_abs", figsize=(18, 4))
plot_receiver_timeseries(data, "csi_phase", figsize=(18, 4))